I - import des données depuis le fichier data.py

In [1]:
from ml_in_finance_ensae.data import load_ff25_and_rf
ff25_excess, rf = load_ff25_and_rf()
ff25_excess.shape, rf.shape

((1194, 25), (1194,))

II - On procède à la régression des 25 portefeuilles sur les facteurs

L'objectif de cette partie est d'obtenir une première version simplifiée (linéaire) de la relation entre les portefeuilles et les facteurs. Quelques points intéressants à remarquer :


On a un modèle de  forme R_i,e,t = alpha_i + Beta_i.T * f_t + eps_i,t.
En posant M_t = 1 − b.⊤ * (f_t​ − μ_f​) on prouve (calculs à rédiger dans le papier) : E[M_t * ​R_i,t,e​] = α_i​.

Autrement dit, l'égalité à 0 recherchée (car M_t en tant que facteur d'actualisation doit vérifier l'égalité E[M_t * ​R_i,t,e​] = 0) s'applique aux α_i.

Dans les régressions ci-dessous, c'est une hypothèse qu'on peut rejeter pour 28% des régressions (on peut réfléchir à faire un test global qui sera rejeté a priori, mais ils n'en font pas vraiment dans le papier). Cela justifie l'idée qu'un modèle linéaire n'est pas suffisant pour capturer l'ensemble des excess returns offerts par le marché.

In [2]:
from ml_in_finance_ensae.data import load_ff25_and_rf, load_ff3_factors, DataPaths

ff25_excess, rf1 = load_ff25_and_rf(DataPaths())
ff3, rf2 = load_ff3_factors(DataPaths())

common = ff25_excess.index.intersection(ff3.index)
print(len(common), common.min(), common.max())
print(ff3.loc[common].head())

1194 1926-07-01 00:00:00 2025-12-01 00:00:00
            Mkt-RF     SMB     HML
Date                              
1926-07-01  0.0289 -0.0255 -0.0239
1926-08-01  0.0264 -0.0114  0.0381
1926-09-01  0.0038 -0.0136  0.0005
1926-10-01 -0.0327 -0.0014  0.0082
1926-11-01  0.0254 -0.0011 -0.0061


In [3]:
from ml_in_finance_ensae.data import load_ff25_and_rf, load_ff3_factors, DataPaths
from ml_in_finance_ensae.bench_ff3 import ff3_time_series_benchmark

ff25_excess, _ = load_ff25_and_rf(DataPaths())
ff3, _ = load_ff3_factors(DataPaths())

res = ff3_time_series_benchmark(ff25_excess, ff3)

res["summary"]
res["table"].sort_values("alpha_m").head(10)


,alpha_m,t_alpha,beta_Mkt-RF,beta_SMB,beta_HML
portfolio,,,,,
SMALL LoBM,-0.006736,-3.330116,1.268709,1.474734,0.350439
ME1 BM2,-0.003939,-3.314086,1.070705,1.529405,0.206005
ME5 BM4,-0.002412,-4.551584,1.031592,-0.179598,0.655390
ME2 BM1,-0.002138,-3.051768,1.085829,1.151121,-0.228246
ME4 BM5,-0.001737,-2.315476,1.185044,0.319525,0.946965
BIG HiBM,-0.001638,-1.632253,1.175891,-0.142561,0.996703
ME1 BM3,-0.001212,-1.405130,1.049362,1.237752,0.492476
ME3 BM1,-0.001190,-2.045613,1.123907,0.810971,-0.218582
ME3 BM5,-0.000621,-0.916865,1.112685,0.589237,0.864848


In [6]:
res["summary"]

{'n_obs': 1194,
 'start': Timestamp('1926-07-01 00:00:00'),
 'end': Timestamp('2025-12-01 00:00:00'),
 'rms_alpha_m': 0.0018472766622951932,
 'rms_alpha_ann': 0.02216731994754232,
 'share_|t_alpha|>2': 0.28}

In [7]:
table = res["table"]

# Ajouter |t_alpha|
table["abs_t_alpha"] = table["t_alpha"].abs()

# Top 5 par |t|
top5 = table.sort_values("abs_t_alpha", ascending=False).head(5)

top5[["alpha_m", "t_alpha"]]


,alpha_m,t_alpha
portfolio,,
ME5 BM4,-0.002412,-4.551584
SMALL LoBM,-0.006736,-3.330116
ME1 BM2,-0.003939,-3.314086
ME2 BM1,-0.002138,-3.051768
BIG LoBM,0.000980,2.981633


Ces résultats sont analysés plus haut.

On est quand même assez éloigné du papier encore ici donc on peut commencer à représenter le modèle d'une manière légèrement différente pour s'en rapprocher.

Hansen–Jagannathan distance : une mesure globale et le pont vers CPZ

La condition de base est toujours : E[M_t * ​R_t,e​] = 0.

Quand elle ne tient pas, on a un vecteur de "pricing errors" :
g = E[M_t * ​R_t,e​]

La question devient : à quel point ce g est grave ?
HJ propose de mesurer ça avec une norme "économiquement pertinente" :

d_HJ(M) =  sqrt(g.T * sigma_R-1 * g) où sigma_R = Var(R_t,e)

Cette quantité a une interprétation : c’est la plus petite dispersion (écart-type) d’un ajustement au SDF nécessaire pour corriger le pricing, dans un certain espace.

CPZ vont dans la même direction, mais au lieu de fixer la norme sigma_R-1, ils construisent une norme adversariale via des instruments g et un problème minimax.


III - Introduction d'une fonction g

On rappelle qu'on fait l'hypothèse que : 
M_t+1 = a - b.T * f_t+1.
Il faut noter que si M_t est un discount facteur admissible (E[M_t * ​R_i,t,e​] = 0) alors quel que soit le scalaire c, c * M_t l'est également.

Aussi, on peut simplement fixer a = 1 et choisir c qui permet de se ramener au M_t précédent (sans perte de généralité), ça simplifie la démarche.

In fine on va donc choisir b (i.e. M) en minimisant une norme de g_hat = 1/T * somme M_t * R_t_e qu'on peut encore écrire (et ce sera le problème à résoudre pour l'instant) :

g_hat(b) = 1/T * somme (t=1:T) de (1 - b.T * f_t) * R_t,e